In [1]:
!pip install numpy==1.16.2

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%env KERAS_BACKED=tensorflow

env: KERAS_BACKED=tensorflow


In [3]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt


In [4]:
#讀入電影數據庫中的影評
from keras.datasets import imdb

/Users/cindy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [44]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=100000)
#只選常用的10萬字，超過這個範圍就當作不存在

In [45]:
print('訓練總比數：',len(x_train))
print('測試總比數：',len(x_test))

訓練總比數： 25000
測試總比數： 25000


## 輸入資料部分

In [46]:
#輸入資料部分
x_train[24999]
#每個數字代表一個文字，資料庫出現頻率越高的文字數字越小ㄩ

[1,
 17,
 6,
 194,
 337,
 7,
 4,
 204,
 22,
 45,
 254,
 8,
 106,
 14,
 123,
 4,
 12815,
 270,
 14437,
 5,
 16923,
 12255,
 732,
 2098,
 101,
 405,
 39,
 14,
 1034,
 4,
 1310,
 9,
 115,
 50,
 305,
 12,
 47,
 4,
 168,
 5,
 235,
 7,
 38,
 111,
 699,
 102,
 7,
 4,
 4039,
 9245,
 9,
 24,
 6,
 78,
 1099,
 17,
 2345,
 16553,
 21,
 27,
 9685,
 6139,
 5,
 29043,
 1603,
 92,
 1183,
 4,
 1310,
 7,
 4,
 204,
 42,
 97,
 90,
 35,
 221,
 109,
 29,
 127,
 27,
 118,
 8,
 97,
 12,
 157,
 21,
 6789,
 85010,
 9,
 6,
 66,
 78,
 1099,
 4,
 631,
 1191,
 5,
 2642,
 272,
 191,
 1070,
 6,
 7585,
 8,
 2197,
 70907,
 10755,
 544,
 5,
 383,
 1271,
 848,
 1468,
 12183,
 497,
 16876,
 8,
 1597,
 8778,
 19280,
 21,
 60,
 27,
 239,
 9,
 43,
 8368,
 209,
 405,
 10,
 10,
 12,
 764,
 40,
 4,
 248,
 20,
 12,
 16,
 5,
 174,
 1791,
 72,
 7,
 51,
 6,
 1739,
 22,
 4,
 204,
 131,
 9]

In [47]:
print('第25000筆評論的長度：',len(x_train[24999]))
print('第1001筆評論的長度：',len(x_train[1000]))
#不同評論長度不同

第25000筆評論的長度： 153
第1001筆評論的長度： 298


## 輸出資料部分
會判斷輸入評論是正評(1) or 負評(0)

In [48]:
y_train[:10] #前十筆評論

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

## 送入神經網路的輸入處理
為了讓資料好訓練，還是會把資料用成一樣長度：

＊設定輸入文字長度的上限

＊把每段文字都弄成一樣長：太短的補上0

In [49]:
from keras.preprocessing import sequence

In [50]:
x_train = sequence.pad_sequences(x_train, maxlen=300)
x_test = sequence.pad_sequences(x_test, maxlen=300)
#設定最長300字，長度太長切斷，太短捕0

In [51]:
x_train.shape

(25000, 300)

## 打造CNN
＊壓縮到n維度

＊用k個LSTM神經元作隱藏層

＊最後ouput用sigmoid輸出（因為是0-1的數字）

In [61]:
n = 10#每個字壓縮到10維
k = 5 #LSTM有5個神經元

In [62]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

In [63]:
model = Sequential() #開啟空的神經網路

In [64]:
model.add(Embedding(100000,n)) #壓縮維度
model.add(LSTM(k))
model.add(Dense(1,activation ='sigmoid'))

## 組裝
用binary_crossentropy做loss function+ Adam學習法

In [67]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [68]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 10)          1000000   
_________________________________________________________________
lstm_5 (LSTM)                (None, 5)                 320       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 1,000,326
Trainable params: 1,000,326
Non-trainable params: 0
_________________________________________________________________


## 訓練

In [69]:
model.fit(x_train,y_train,
         batch_size=32,
         epochs=2)

Epoch 1/2
25000/25000 [==============================] - 171s 7ms/step - loss: 0.4912 - acc: 0.7756
Epoch 2/2
25000/25000 [==============================] - 169s 7ms/step - loss: 0.2360 - acc: 0.9150


In [70]:
score = model.evaluate(x_test,y_test)
print('loss:',score[0])
print('正確率:',score[1])

25000/25000 [==============================] - 20s 782us/step
loss: 0.33816872793197633
正確率: 0.86244


### 上課模型正確率：0.85152

## 多次調整後最佳值：0.86224

In [71]:
x_train = sequence.pad_sequences(x_train, maxlen=500)
x_test = sequence.pad_sequences(x_test, maxlen=500)
n = 10 #每個字壓縮到二十維
k = 10 #LSTM有10個神經元
model = Sequential() #開啟空的神經網路
model.add(Embedding(100000,n)) 
model.add(LSTM(k))
model.add(Dense(1,activation ='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train,y_train,
         batch_size=32,
         epochs=2)

Epoch 1/2
25000/25000 [==============================] - 283s 11ms/step - loss: 0.4588 - acc: 0.7833
Epoch 2/2
25000/25000 [==============================] - 282s 11ms/step - loss: 0.2174 - acc: 0.9222


In [72]:
score = model.evaluate(x_test, y_test)
print(f'測試資料的 loss = {score[0]}')
print(f'測試資正確率 = {score[1]}')

25000/25000 [==============================] - 35s 1ms/step
測試資料的 loss = 0.31914688110351563
測試資正確率 = 0.86824


## 再調整k與maxlen，增加至0.86824

In [73]:
model.fit(x_train,y_train,
         batch_size=32,
         epochs=2)

Epoch 1/2
25000/25000 [==============================] - 283s 11ms/step - loss: 0.1193 - acc: 0.9608
Epoch 2/2
25000/25000 [==============================] - 282s 11ms/step - loss: 0.0747 - acc: 0.9785


In [74]:
score = model.evaluate(x_test, y_test)
print(f'測試資料的 loss = {score[0]}')
print(f'測試資正確率 = {score[1]}')

25000/25000 [==============================] - 35s 1ms/step
測試資料的 loss = 0.41846757267713547
測試資正確率 = 0.86056


增加訓練次數沒有效果

### 調整了num_words/壓縮後維度/LSTM神經元數/訓練次數的最高正確率：86.824%